In [35]:
import scipy.io
import tensorflow as tf
import numpy as np
from tensorflow import  keras
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten

In [2]:
def load_data():
  als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["data"].transpose()  #(262134, 321)
  normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['data'].transpose() # (262134, 270) 
  myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['data'].transpose() #(262134, 315)
  return als,normal,myopathie

In [3]:

def scale_data_minmax(data):
  scaler = MinMaxScaler()
  normalized_data =scaler.fit_transform(data)


  return normalized_data

In [4]:
def scale_data_standard(data):
  scaled_data = scale(data,axis=1)
  return scaled_data

In [5]:
def gen_tensors_list(data,dim):
  dataset  = list()
  for i in range (len(data)):
    arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
    arg = tf.reshape(arg,dim)
    dataset.append(arg)
  return dataset

In [6]:
def windowing_fun(data,window_size=1000,overlap=100):
  windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
  final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
  return final_data

In [7]:
def load_file_names():
  als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["files"]  #(262134, 321)
  normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['files'] # (262134, 270) 
  myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['files'] #(262134, 315)
  return als,normal,myopathie

In [8]:
def generate_dataframe(data,files):
  df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
  windowed_data = [{"signal": data[j],"type_signal":files[0][j][0][5],"num_personne": files[0][j][0][6:8],"muscle": files[0][j][0][8:10],"num_enregistrement":files[0][j][0][10:12] } for j in range(0,len(data)) ]
  final_data = [windowed_data[i] for i in range(len(windowed_data))  ]
  df =df.append(final_data, ignore_index=True)
  return df

In [9]:
als, normal , myopathie = load_data()


In [10]:
alsfiles,normalfiles,myopathiefiles = load_file_names()

In [11]:
df = generate_dataframe(np.vstack((normal,als,myopathie)),np.concatenate ((normalfiles,alsfiles,myopathiefiles),axis=1))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13576\3882924509.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df.append(final_data, ignore_index=True)


In [12]:
def f(row):
  return {"C":0,"A":1,"M":2}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [13]:
df = df.query("muscle=='BB'")

In [14]:
X = df['signal'].tolist()
Y = df['num_class'].tolist()

In [15]:
X=X[170:]
Y=Y[170:]

In [16]:
del df

In [17]:
data = [*X]
data, np.shape(data)

([array([-1.90737774, -0.38147555, -2.05996796, ..., 10.22354467,
         13.88570993, 11.13908598]),
  array([ -4.8065919 , -14.03830014, -23.27000839, ...,  33.26466773,
          25.94033722,  28.83955138]),
  array([22.5070573 , 23.49889372, 22.20187686, ..., 23.95666438,
         19.15007248, 24.33813993]),
  array([ -4.57770657, -10.91020066, -13.04646372, ..., -42.72526131,
         -45.62447547, -44.40375372]),
  array([14.57236591, 15.2590219 , 14.87754635, ..., 19.45525292,
         15.48790723, 17.39528496]),
  array([40.81788357, 45.39559014, 41.96231022, ...,  0.8392462 ,
          0.76295109,  2.59403372]),
  array([ 6.25619898,  6.56137942,  4.27252613, ..., -5.6458381 ,
         -4.42511635, -3.81475547]),
  array([-18.5397116 , -17.62417029, -18.61600671, ...,   5.34065766,
           6.94285496,   4.11993591]),
  array([-61.18867781, -60.50202182, -60.12054627, ..., -27.23735409,
         -28.30548562, -28.53437095]),
  array([ 1.83108263,  2.97550927,  2.59403372, .

In [18]:
del  als,normal,myopathie,alsfiles,normalfiles,myopathiefiles 

In [19]:
data = scale_data_standard(data)

In [20]:
last_index_normal = len(Y) - 1 - Y[::-1].index(0)

In [24]:
last_index_als = len(Y) - 1 - Y[::-1].index(1)

In [25]:
last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)

In [26]:
windowed_normal = len(windowing_fun(X[:last_index_normal+1],10000))
windowed_als = len(windowing_fun(X[last_index_normal+1:last_index_als+1],10000))
windowed_myopathie = len(windowing_fun(X[last_index_als+1:],10000))

In [27]:
Y = [0]*(windowed_normal)+[1]*(windowed_als)+[2]*(windowed_myopathie)

In [28]:
Y  = Y*1

In [29]:
X = windowing_fun(data,10000)

In [30]:
del data

In [31]:
X = scale_data_standard(X)

In [32]:
tensors_list = gen_tensors_list(X,(1,10000))

In [43]:
X_train, X_test, y_train, y_test = train_test_split( tensors_list, Y, test_size=0.2,random_state=20)
np.shape(X_train) , np.shape(X_test)

((6344, 1, 10000), (1586, 1, 10000))

In [41]:
model = Sequential([
    #Conv2D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(64, 3, activation = 'relu', input_shape = (10000,1,6344)), 
    
    #MaxPooling2D(filter size stride and pad can also be added)
    MaxPooling1D(2, strides = 2),
    
    #LAYER2
    Conv1D(32, 3, activation = 'relu'),
    MaxPooling1D(2, strides = 2),
    
    #FLATTEN
    Flatten(),
    
    #FC Layers:
    
    #LAYER3
    #Dense : fully connected  Dense(#neurons, activation function )
    Dense(64, activation = 'relu'),
    
    #LAYER4
    Dense(3, activation = 'softmax')
    
    
])


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_6. Consider increasing the input size. Received input shape [None, 10000, 1, 6344] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/x_train.npy', 'wb') as f:
    np.save(f, np.asarray(X_train))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/x_test.npy', 'wb') as f:
    np.save(f, np.asarray(X_test))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/y_train.npy', 'wb') as f:
    np.save(f, np.asarray(y_train))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/y_test.npy', 'wb') as f:
    np.save(f, np.asarray(y_test))